In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome import service as fs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup

import requests

from collections import Counter
import os
import time
import re

In [ ]:
# open chrome
chrome_driver = '../driver/chromedriver'
chrome_service = fs.Service(chrome_driver)
driver = webdriver.Chrome(service=chrome_service)

In [ ]:
image_file = 'sample_item.jpg'
keywords = search_keywords_by_image(image_file)
driver.quit()

In [ ]:
def search_keywords_by_image(image_file):

    # execute image search
    driver.get('https://images.google.com/')
    cam_button = driver.find_elements(By.CLASS_NAME, 'nDcEnd')
    cam_button[0].click()

    delay = 5 # seconds
    element = (By.CLASS_NAME, 'DV7the')
    wait_until_page_is_loaded(delay, element)

    upload_btn = driver.find_element(By.NAME, 'encoded_image')
    upload_btn.send_keys(os.getcwd() + '/' + image_file)

    element = (By.CLASS_NAME, 'TfLgNb')
    wait_until_page_is_loaded(delay, element)

    # obtain search result url
    url = driver.current_url

    keywords = extract_keywords(url)

    return keywords

def wait_until_page_is_loaded(delay, element):

    try:
        WebDriverWait(driver, delay).until(EC.presence_of_element_located(element))
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")

def extract_keywords(url):

    # extract text data
    html = requests.get(url)
    soup = BeautifulSoup(html.text, features='lxml')
    scripts = soup.select('script')
    matched_images_data = ''.join(re.findall(r"AF_initDataCallback\(([^<]+)\);", str(scripts)))
    extracted_text = re.findall('"([^"]*)"', matched_images_data)

    # extract keywords
    texts = ' '.join(extracted_text).split(' ')
    texts = [text for text in texts if len(text) > 1]
    texts = Counter(texts)
    keywords = [word[0] for word in texts.most_common(20)]

    return keywords